<a href="https://colab.research.google.com/github/chongzicbo/nlp-ml-dl-notes/blob/master/code/textclassification/text_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 基于TextCnn的中文文本分类


In [3]:
import torch
from torch import nn
from torch.nn import functional as F
import math
from torch.utils.data import Dataset,DataLoader
import numpy as np
import random
from sklearn.model_selection import train_test_split
import pandas as pd
import re
from tensorflow.keras.preprocessing import sequence

# 1.数据预处理及准备训练和测试数据

In [4]:
maxlen=100
batch_size=128

## 1.1 文本预处理

In [5]:
def textToChars(filePath):
    """
    读取文本文件并进行处理
    :param filePath:文件路径
    :return:
    """
    lines = []
    df=pd.read_excel(filePath,header=None)
    df.columns=['content']
    for index, row in df.iterrows():
      row=row['content']
      row = re.sub("[^\u4e00-\u9fa5]", "", str(row))  # 只保留中文
      lines.append(list(row))
    return lines


def getWordIndex(vocabPath):
    """
    获取word2Index,index2Word
    :param vocabPath:词汇文件
    :return:
    """
    word2Index = {}
    with open(vocabPath, encoding="utf-8") as f:
        for line in f.readlines():
            word2Index[line.strip()] = len(word2Index)
    index2Word = dict(zip(word2Index.values(), word2Index.keys()))
    return word2Index, index2Word


def lodaData(posFile, negFile, word2Index):
  """
  获取训练数据
  :param posFile:正样本文件
  :param negFile:负样本文件
  :param word2Index:
  :return:
  """
  posLines = textToChars(posFile)
  negLines = textToChars(negFile)
  posIndexLines = [[word2Index[word] if word2Index.get(word) else 0 for word in line] for line in posLines]
  negIndexLines = [[word2Index[word] if word2Index.get(word) else 0 for word in line] for line in negLines]
  lines = posIndexLines + negIndexLines
  print("训练样本和测试样本共：%d 个"%(len(lines)))
  # lens = [len(line) for line in lines]
  labels = [1] * len(posIndexLines) + [0] * len(negIndexLines)
  padSequences = sequence.pad_sequences(lines, maxlen=maxlen, padding="post", truncating="post")
  X_train,X_test,y_train,y_test=train_test_split(padSequences,np.array(labels),test_size=0.2,random_state=42)
  return X_train,X_test,y_train,y_test

In [6]:
vocabPath="/content/drive/My Drive/data/vocab.txt"
negFilePath="/content/drive/My Drive/data/text_classify/sentiment/neg.xls"
posFilePath="/content/drive/My Drive/data/text_classify/sentiment/pos.xls"
word2Index, index2Word=getWordIndex(vocabPath)
X_train,X_test,y_train,y_test=lodaData(posFile=posFilePath,negFile=negFilePath,word2Index=word2Index)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

训练样本和测试样本共：21105 个
(16884, 100) (4221, 100) (16884,) (4221,)


## 1.2  自定义Dataset

In [7]:
class MyDataset(Dataset):

  def __init__(self,features,labels):
    """
    features:文本向量化后的特征
    labels:标签向量 
    """
    self.features=features
    self.labels=labels

  def __len__(self):
    return self.features.shape[0]

  def __getitem__(self,index):
    return self.features[index],self.labels[index]

    
train_dataset=MyDataset(X_train,y_train)
test_dataset=MyDataset(X_test,y_test)
train_dataloader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_dataloader=DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

In [8]:
for features,labels in train_dataloader:
  print(features.shape,labels.shape)
  break

torch.Size([128, 100]) torch.Size([128])


# 2.网络搭建

In [9]:
class TextCnn(nn.Module):
  def __init__(self, param: dict):
    super(TextCnn, self).__init__()
    input_channel = 1  # input channel size
    output_channel = param["output_channel"]  # output channel size
    kernel_size = param["kernel_size"]
    vocab_size = param["vocab_size"]
    embedding_dim = param["embedding_dim"]
    dropout = param["dropout"]
    class_num = param["class_num"]
    self.param = param
    self.embedding = nn.Embedding(vocab_size, embedding_dim,padding_idx=0)
    self.conv1 = nn.Conv2d(input_channel, output_channel, (kernel_size[0], embedding_dim))
    self.conv2 = nn.Conv2d(input_channel, output_channel, (kernel_size[1], embedding_dim))
    self.conv3 = nn.Conv2d(input_channel, output_channel, (kernel_size[2], embedding_dim))
    self.dropout = nn.Dropout(dropout)
    self.fc1 = nn.Linear(len(kernel_size) * output_channel, class_num)

  def init_embedding(self, embedding_matrix):
    self.embedding.weight = nn.Parameter(torch.Tensor(embedding_matrix))

  @staticmethod
  def conv_pool(x, conv):
    """
    卷积+池化
    :param x:[batch_size,1,sequence_length,embedding_dim]
    :param conv:
    :return:
    """
    x = conv(x)  # 卷积， [batch_size,output_channel,h_out,1]
    x = F.relu((x.squeeze(3)))  # 去掉最后一维,[batch_size,output_channel,h_out]
    x = F.max_pool1d(x, x.size(2)).squeeze(2)  # [batch_size,output_channel]
    return x

  def forward(self, x):
    """
    前向传播
    :param x:[batch_size,sequence_length]
    :return:
    """
    x = self.embedding(x)  # [batch_size,sequence_length,embedding_dim]
    x = x.unsqueeze(1)  # 增加一个channel维度 [batch_size,1,sequence_length,embedding_dim]
    x1 = self.conv_pool(x, self.conv1)  # [batch_size,output_channel]
    x2 = self.conv_pool(x, self.conv2)  # [batch_size,output_channel]
    x3 = self.conv_pool(x, self.conv3)  # [batch_size,output_channel]
    x = torch.cat((x1, x2, x3), 1)  # [batch_size,output_channel*3]
    x = self.dropout(x)
    logit = F.log_softmax(self.fc1(x), dim=1)
    return logit

  def init_weights(self):
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        m.weight.data.normal_(0, math.sqrt(2. / n))
        if m.bias is not None:
          m.bias.data.zero_()

      elif isinstance(m, nn.BatchNorm2d):
        m.weight.data.fill_(1)
        m.bias.data.zero_()
      elif isinstance(m, nn.Linear):
        m.weight.data.normal_(0, 0.01)
        m.bias.data.zero_()


# 3.模型训练

In [19]:
textCNNParams={
    "vocab_size":len(word2Index),
    "embedding_dim":200,
    "class_num":2,
    "output_channel":16,
    "kernel_size":[3,4,5],
    "dropout":0.2
}

In [20]:
net=TextCnn(textCNNParams)
net.init_weights()

In [21]:
net.cuda()

TextCnn(
  (embedding): Embedding(21128, 200, padding_idx=0)
  (conv1): Conv2d(1, 16, kernel_size=(3, 200), stride=(1, 1))
  (conv2): Conv2d(1, 16, kernel_size=(4, 200), stride=(1, 1))
  (conv3): Conv2d(1, 16, kernel_size=(5, 200), stride=(1, 1))
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=48, out_features=2, bias=True)
)

In [23]:
optimizer=torch.optim.Adam(net.parameters(),lr=0.01)
criterion=nn.NLLLoss()

In [24]:
for epoch in range(50):
  total_train_loss=[]
  net.train()
  for i,(feature,label) in enumerate(train_dataloader):
    feature=feature.cuda()
    label=label.cuda()
    y_pred=net(feature.long()) #前向计算
    loss=criterion(y_pred,label) #计算损失
    optimizer.zero_grad() #清除梯度
    loss.backward() #计算梯度，误差回传
    optimizer.step() #更新参数
    total_train_loss.append(loss.data.item())
  total_valid_loss=[]
  pred_true_labels=0
  net.eval()
  for i,(feature_test,label_test) in enumerate(test_dataloader):
    feature_test=feature_test.cuda()
    label_test=label_test.cuda()
    with torch.no_grad():
      pred_test=net(feature_test.long())
      test_loss=criterion(pred_test,label_test)
      total_valid_loss.append(test_loss.data.item())
      # accu=torch.sum((torch.argmax(pred_test,dim=1)==label_test)).data.item()/feature_test.shape[0]
      pred_true_labels+=torch.sum(torch.argmax(pred_test,dim=1)==label_test).data.item()
      
  print("epoch:{},  train_loss:{:.3f},  test_loss:{:.3f},  test accuracy:{:.3f}".format(epoch,np.mean(total_train_loss),np.mean(total_valid_loss),pred_true_labels/len(test_dataset)))

epoch:0,  train_loss:0.475,  test_loss:0.299,  test accuracy:0.879
epoch:1,  train_loss:0.299,  test_loss:0.265,  test accuracy:0.894
epoch:2,  train_loss:0.211,  test_loss:0.326,  test accuracy:0.889
epoch:3,  train_loss:0.162,  test_loss:0.273,  test accuracy:0.905
epoch:4,  train_loss:0.116,  test_loss:0.353,  test accuracy:0.905
epoch:5,  train_loss:0.105,  test_loss:0.362,  test accuracy:0.910
epoch:6,  train_loss:0.097,  test_loss:0.347,  test accuracy:0.911
epoch:7,  train_loss:0.079,  test_loss:0.487,  test accuracy:0.904
epoch:8,  train_loss:0.089,  test_loss:0.489,  test accuracy:0.896
epoch:9,  train_loss:0.076,  test_loss:0.778,  test accuracy:0.878
epoch:10,  train_loss:0.096,  test_loss:0.650,  test accuracy:0.900
epoch:11,  train_loss:0.077,  test_loss:0.635,  test accuracy:0.901
epoch:12,  train_loss:0.079,  test_loss:0.698,  test accuracy:0.903
epoch:13,  train_loss:0.063,  test_loss:0.780,  test accuracy:0.896
epoch:14,  train_loss:0.072,  test_loss:0.764,  test accur

# 4.模型测试

In [15]:
def predict_one(sentence,net,word2Index):
  sentence=re.sub("[^\u4e00-\u9fa5]", "", str(sentence))  # 只保留中文
  print(sentence)
  sentence=[word2Index[word] if word2Index.get(word) else 0 for word in sentence]
  sentence=sentence+[0]*(maxlen-len(sentence)) if len(sentence)<maxlen else sentence[0:300]
  print(sentence)
  sentence=torch.tensor(np.array(sentence)).view(-1,len(sentence)).cuda()
  label=torch.argmax(net(sentence),dim=1).data.item()
  print(label)


In [16]:
sentence="一次很不爽的购物，页面上说是第二天能到货，结果货是从陕西发出的，卖家完全知道第二天根本到不了货。多处提到送货入户还有100%送货入户也没有兑现，与客服联系多日，还是把皮球踢到快递公司。算是一个教训吧。"
predict_one(sentence,net,word2Index)

一次很不爽的购物页面上说是第二天能到货结果货是从陕西发出的卖家完全知道第二天根本到不了货多处提到送货入户还有送货入户也没有兑现与客服联系多日还是把皮球踢到快递公司算是一个教训吧
[671, 3613, 2523, 679, 4272, 4638, 6579, 4289, 7552, 7481, 677, 6432, 3221, 5018, 753, 1921, 5543, 1168, 6573, 5310, 3362, 6573, 3221, 794, 7362, 6205, 1355, 1139, 4638, 1297, 2157, 2130, 1059, 4761, 6887, 5018, 753, 1921, 3418, 3315, 1168, 679, 749, 6573, 1914, 1905, 2990, 1168, 6843, 6573, 1057, 2787, 6820, 3300, 6843, 6573, 1057, 2787, 738, 3766, 3300, 1050, 4385, 680, 2145, 3302, 5468, 5143, 1914, 3189, 6820, 3221, 2828, 4649, 4413, 6677, 1168, 2571, 6853, 1062, 1385, 5050, 3221, 671, 702, 3136, 6378, 1416, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0
